In [ ]:
from final_training.gen_vs_pers import *

In [ ]:
# Simulazione degli argomenti per il notebook
class Args:
    def __init__(self):
        self.gru_results_file = "outputs/test_set/gru_output.csv"
        self.xgb_results_file = "outputs/test_set/xgb_output.csv"
        self.output_dir = "outputs/personalized"
        self.models_dir = "models/personalized"
        self.scores_dir = "scores/gen_vs_pers"
        self.plots_dir = "plots/gen_vs_pers"
        self.test_split = 0.2
        self.seed = 42
        self.epochs = 100
        self.batch_size = 256
        self.lr = 0.01
        self.es_patience = 20


args = Args()

# Verifica i parametri
print(f"Configurazione:")
print(f"  - GRU results file: {args.gru_results_file}")
print(f"  - XGB results file: {args.xgb_results_file}")
print(f"  - Output dir: {args.output_dir}")
print(f"  - Models dir: {args.models_dir}")
print(f"  - Scores dir: {args.scores_dir}")
print(f"  - Plots dir: {args.plots_dir}")
print(f"  - Test split: {args.test_split}")
print(f"  - Seed: {args.seed}")
print(f"  - Epochs: {args.epochs}")
print(f"  - Batch size: {args.batch_size}")
print(f"  - Learning rate: {args.lr}")
print(f"  - Early stopping patience: {args.es_patience}")

In [ ]:
print("🚀 CONFRONTO MODELLI GENERALIZZATI VS PERSONALIZZATI")
print("=" * 60)

# Setup environment
setup_environment(args)

In [ ]:
# Identifica pazienti estremi
best_patient, worst_patient = identify_extreme_patients(args.gru_results_file)
target_patients = [best_patient, worst_patient]

print(f"Target patients: {target_patients}")

In [ ]:
# Carica dati originali
all_data, X_cols, y_cols = load_original_data()

print(f"Features: {len(X_cols)} columns")
print(f"Target: {y_cols}")

In [ ]:
# Raccoglitore per tutte le metriche
all_metrics = []

# Processa ogni paziente target
for patient_id in target_patients:
    print(f"\n" + "=" * 50)
    print(f"PROCESSING PAZIENTE {patient_id}")
    print("=" * 50)

    # Estrai dati del paziente
    patient_data = all_data[all_data["Patient_ID"] == patient_id].copy()
    print(f"Dati totali paziente {patient_id}: {len(patient_data)} campioni")

    # Crea split temporali per addestramento modelli personalizzati
    train_data, test_data = create_patient_splits(
        patient_data, args.test_split, patient_id
    )

    # Addestra modelli personalizzati
    xgb_personalized = train_personalized_xgb(
        train_data, X_cols, y_cols, patient_id, args
    )
    gru_personalized = train_personalized_gru(
        train_data, test_data, X_cols, y_cols, patient_id, args
    )

    # Valuta modelli personalizzati
    personalized_results = evaluate_personalized_models(
        patient_id,
        test_data,
        X_cols,
        y_cols,
        xgb_personalized,
        gru_personalized,
        args,
    )

    # Carica risultati modelli generalizzati
    generalized_results = load_generalized_results(
        patient_id, args.xgb_results_file, args.gru_results_file
    )

    # Calcola metriche comparative
    patient_metrics = calculate_comparison_metrics(
        patient_id, personalized_results, generalized_results, args
    )

    if patient_metrics is not None:
        all_metrics.append(patient_metrics)

    print(f"✓ Completato processamento paziente {patient_id}")

In [ ]:
# Combina tutte le metriche
if all_metrics:
    all_metrics_df = pd.concat(all_metrics, ignore_index=True)

    # Crea riassunto finale
    summary_df = create_comparison_summary(all_metrics_df, args)

    print(f"\n✅ Confronto completato! Risultati salvati in: {args.output_dir}")

    results = {
        "metrics": all_metrics_df,
        "summary": summary_df,
        "target_patients": target_patients,
    }
else:
    print("\n❌ Nessun risultato generato")
    results = None